크롤러 설명:
    주식에 있어 거래량은 중요한 투자판매지표가 될 수 있다.
    거래량이 많다는 것은 그만큼 사람들의 수요와 관심이 많은 핫한 종목이라는
    뜻이기도 하다. 네이버증권(https://m.stock.naver.com)에서 코스피, 코스닥의 거래상위 종목(100)을 얻어내고,
    해당 종목 100개에 대한 뉴스들을 크롤링해 거래량순위(txt,csv,xlsx), 각 종목(100개)에 대한 뉴스파일(txt,csv,xlsx)을 생성할 것이다.

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import sys        #system 설정을 변경하기 위해 필요
import os
import numpy           
import pandas  as pd 
import math  #페이지 번호 계산을 위해
import keyboard

In [2]:
os.chdir('D:\\ai\\소활코\\homework') # 실행후 저장소가 바뀌는것 방지.

In [3]:
now = time.localtime()
txt1='Trading Volume_'
txt2=' %04d-%02d-%02d-%02d-%02d-%02d' %(now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)
dir_name = txt1+txt2
os.mkdir(dir_name)
time.sleep(0.3)
os.chdir('D:\\ai\\소활코\\homework\\'+dir_name)

In [4]:
choice=input('코스피와 코스닥 둘 중 어느것을 탐색하시겠습니까: ')
fix='D:\\ai\\소활코\\homework\\'+dir_name
if choice=='코스피': #코스피인지 코스닥인지에 따른 디렉토리명을 위함
    name='kospi'
else:
    name='kosdaq'
kospi_volume_ft=fix+"\\"+name+"_volume"+txt2+".txt"
kospi_volume_fc=fix+"\\"+name+"_volume"+txt2+".csv"
kospi_volume_fx=fix+"\\"+name+"_volume"+txt2+".xlsx"
path="C:/Users/Aiden/chromedriver_86/chromedriver.exe"
driver=webdriver.Chrome(path)
s_time = time.time( )      #크롤링 시작 시간을 위한 타임 스탬프 
driver.get("https://m.stock.naver.com")
time.sleep(2) #위 페이지가 모두 열릴때까지 2초 기다린다

코스피와 코스닥 둘 중 어느것을 탐색하시겠습니까: 코스피


In [5]:
driver.find_element_by_link_text("국내").click()

In [6]:
driver.find_element_by_link_text("거래상위").click()

In [7]:
if name=='kosdaq':
    driver.find_element_by_link_text("코스닥").click()
else:
    driver.find_element_by_link_text("코스피").click()

In [8]:
#스크롤 한번 안내려주면 에러나길래 스크롤을 한번 끝까지 내려주는 코드
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(0.2)
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(0.2)
#더 보기 2개 눌러서 상위 100개 전체 나타나게 바꿈
driver.find_element_by_xpath("""//*[@id="content"]/div/div[2]/div[2]/div[3]/a/div/span/span""").click()
time.sleep(0.5)
driver.find_element_by_xpath("""//*[@id="content"]/div/div[2]/div[2]/div[3]/a/div/span/span""").click()

In [9]:
trading_volume=[]
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
volume_chart=soup.select('tbody>tr')

In [10]:
stock_name=[] #종목명
stock_price=[] #시세
stock_yesterday=[] # 전일비
stock_updownrate=[] # 등락률
stock_size=[] # 시가총액(억)
stock_tradingvolume=[] #거래량

table = driver.find_element_by_class_name('item_tbl')
tbody = table.find_element_by_tag_name("tbody")
rows = tbody.find_elements_by_tag_name("tr")
for index, value in enumerate(rows):
    name=value.find_elements_by_tag_name("td")[0]
    stock_name.append(name.text)
    price=value.find_elements_by_tag_name("td")[1]
    stock_price.append(price.text)
    yesterday=value.find_elements_by_tag_name("td")[2]
    stock_yesterday.append(yesterday.text)
    updownrate=value.find_elements_by_tag_name("td")[3]
    stock_updownrate.append(updownrate.text)
    size=value.find_elements_by_tag_name("td")[4]
    stock_size.append(size.text)
    tradingvolume=value.find_elements_by_tag_name("td")[5]
    stock_tradingvolume.append(tradingvolume.text)

In [11]:
stock_name

['KODEX 200선물인버스2X',
 'KODEX 코스닥150선물인버스',
 '이아이디',
 'KODEX 레버리지',
 '까뮤이앤씨',
 '신한 레버리지 WTI원유 선물 ETN(H)',
 '삼성 레버리지 WTI원유 선물 ETN',
 'KODEX 코스닥150 레버리지',
 '남성',
 'KODEX 인버스',
 '알루코',
 '삼성엔지니어링',
 '삼성전자',
 '영흥',
 '신성이엔지',
 'KEC',
 '써니전자',
 '신성통상',
 'KODEX 200',
 'KODEX 코스닥 150',
 'HMM',
 '수산중공업',
 '카프로',
 '아남전자',
 '동양물산',
 'KODEX WTI원유선물(H)',
 '한솔홈데코',
 '파미셀',
 '삼부토건',
 '제일약품',
 '미래에셋 레버리지 원유선물혼합 ETN(H)',
 'TCC스틸',
 '진원생명과학',
 '한화생명',
 '사조씨푸드',
 '한화솔루션',
 '형지엘리트',
 '금호에이치티',
 '대창',
 '삼성 인버스 2X WTI원유 선물 ETN',
 '신한지주',
 '진도',
 '세진중공업',
 '삼성중공업',
 '두산중공업',
 '삼성전자우',
 'TIGER 200',
 '태림포장',
 '대한전선',
 '한국전력',
 'TIGER 코스닥150선물인버스',
 'KODEX 단기채권PLUS',
 '두산인프라코어',
 'TIGER 200선물인버스2X',
 'SK하이닉스',
 '체시스',
 '기아차',
 '콤텍시스템',
 'TIGER 원유선물Enhanced(H)',
 '우리금융지주',
 '기업은행',
 '서울식품',
 '대우건설',
 'SK증권',
 'BNK금융지주',
 'LG디스플레이',
 'KB금융',
 '아시아나항공',
 '비티원',
 '남선알미늄',
 '하나금융지주',
 'CJ씨푸드',
 '오리엔트바이오',
 'TIGER 200선물레버리지',
 '미래에셋대우',
 '팬오션',
 '에이프로젠제약',
 '이수화학',
 '메리츠증권',
 'KBSTAR 단기통안채',
 'DGB금융지주',
 '모나미',
 '보락',

In [12]:
no=1
for i in range(100):
    f=open(kospi_volume_ft,'a',encoding='UTF-8')
    print("●"+str(no)+"순위")
    f.write(str(no)+"순위"+"\n")
    #종목명 가져오기
    f.write("\t1. 종목명: "+stock_name[i]+"\n")
    print("\t1. 종목명: "+stock_name[i])
    #시세가져오기
    f.write("\t2. 시세: "+stock_price[i]+"\n")
    print("\t2. 시세: "+stock_price[i])
    #전일비 가져오기
    f.write("\t3. 전일비: "+stock_yesterday[i]+"\n")
    print("\t3. 전일비: "+stock_yesterday[i])
    #등락율 가져오기
    f.write("\t4. 등락률: "+stock_updownrate[i]+"\n")
    print("\t4. 등락률: "+stock_updownrate[i])    
    #시총가져오기
    f.write("\t5. 시총(억): "+stock_size[i]+"\n")
    print("\t5. 시총(억): "+stock_size[i]) 
    #거래량 가져오기
    f.write("\t6. 거래량: "+stock_tradingvolume[i]+"\n")
    print("\t6. 거래량: "+stock_tradingvolume[i]) 
    no+=1
#print(trading_volume)

●1순위
	1. 종목명: KODEX 200선물인버스2X
	2. 시세: 4,410
	3. 전일비: 상승220
	4. 등락률: +5.25%
	5. 시총(억): 25,066
	6. 거래량: 212,784,340
●2순위
	1. 종목명: KODEX 코스닥150선물인버스
	2. 시세: 5,700
	3. 전일비: 상승215
	4. 등락률: +3.92%
	5. 시총(억): 6,686
	6. 거래량: 120,502,546
●3순위
	1. 종목명: 이아이디
	2. 시세: 274
	3. 전일비: 상승35
	4. 등락률: +14.64%
	5. 시총(억): 1,516
	6. 거래량: 117,202,177
●4순위
	1. 종목명: KODEX 레버리지
	2. 시세: 13,860
	3. 전일비: 하락765
	4. 등락률: -5.23%
	5. 시총(억): 20,374
	6. 거래량: 69,650,657
●5순위
	1. 종목명: 까뮤이앤씨
	2. 시세: 2,415
	3. 전일비: 상승375
	4. 등락률: +18.38%
	5. 시총(억): 1,090
	6. 거래량: 68,987,399
●6순위
	1. 종목명: 신한 레버리지 WTI원유 선물 ETN(H)
	2. 시세: 220
	3. 전일비: 하락25
	4. 등락률: -10.20%
	5. 시총(억): 1,056
	6. 거래량: 58,826,529
●7순위
	1. 종목명: 삼성 레버리지 WTI원유 선물 ETN
	2. 시세: 260
	3. 전일비: 하락30
	4. 등락률: -10.34%
	5. 시총(억): 1,050
	6. 거래량: 57,223,718
●8순위
	1. 종목명: KODEX 코스닥150 레버리지
	2. 시세: 10,580
	3. 전일비: 하락880
	4. 등락률: -7.68%
	5. 시총(억): 10,485
	6. 거래량: 49,402,423
●9순위
	1. 종목명: 남성
	2. 시세: 4,145
	3. 전일비: 상한가955
	4. 등락률: +29.94%
	5. 시총(억): 1,501
	6. 거래량: 40,564,482
●10순위
	1

In [13]:
#pandas 활용하여 데이터 프레임 생성
stock_volume_chart = pd.DataFrame()
stock_volume_chart['종목명'] = stock_name
stock_volume_chart['시세'] = stock_price
stock_volume_chart['전일비'] = stock_yesterday
stock_volume_chart['등락률'] = stock_updownrate
stock_volume_chart['시총(억)'] = stock_size
stock_volume_chart['거래량'] = stock_tradingvolume

In [14]:
print(stock_volume_chart)

                        종목명      시세    전일비      등락률   시총(억)          거래량
0          KODEX 200선물인버스2X   4,410  상승220   +5.25%  25,066  212,784,340
1         KODEX 코스닥150선물인버스   5,700  상승215   +3.92%   6,686  120,502,546
2                      이아이디     274   상승35  +14.64%   1,516  117,202,177
3                KODEX 레버리지  13,860  하락765   -5.23%  20,374   69,650,657
4                     까뮤이앤씨   2,415  상승375  +18.38%   1,090   68,987,399
..                      ...     ...    ...      ...     ...          ...
95                       KT  22,100  하락500   -2.21%  57,706    1,307,586
96  QV 레버리지 WTI원유 선물 ETN(H)     215   하락25  -10.42%      43    1,261,302
97                     대한항공  19,950  하락350   -1.72%  34,755    1,242,695
98                      디피씨   9,460  하락590   -5.87%   3,943    1,226,900
99                       동방   1,895   하락75   -3.81%     706    1,210,014

[100 rows x 6 columns]


In [15]:
#csv 형태로 저장하기
stock_volume_chart.to_csv(kospi_volume_fc,encoding="utf-8-sig",index=False)

#엑셀 형태로 저장하기
stock_volume_chart.to_excel(kospi_volume_fx,index=False)

종목별 뉴스 크롤링

In [16]:
element = driver.find_element_by_class_name('Nbox_input_text')
element.send_keys(stock_name[0])
time.sleep(0.5)
driver.find_element_by_class_name('Nicon_submit').click()
index=1
no2=[]           # 뉴스 번호 컬럼
title2=[]       # 뉴스 제목 컬럼
contents2=[]    # 뉴스 내용 컬럼
news2=[] # 발행처
date2=[] # 뉴스 날짜
#페이지 양식이 첫페이지만 달라서 1순위는 따로돌리고 나머지 99번 반복

In [17]:
driver.find_element_by_link_text('뉴스·공시').click()

In [18]:
#test
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
news_list = soup.select('.title')
news_paragraph=soup.select('.news_ct')
news_maker=soup.select('.press')
news_date=soup.select('em',class_='time')

In [19]:
for title in news_list:
    title2.append(title.get_text())
for para in news_paragraph:
    contents2.append(para.get_text())
for maker in news_maker:
    news2.append(maker.get_text())
for date in news_date:
    date2.append(date.get_text())

In [20]:
date2

['',
 '상승',
 '',
 '장마감',
 '2020.10.11',
 '2020.09.27',
 '2020.08.09',
 '2020.08.02',
 '2020.07.16',
 '2020.07.14',
 '2020.07.12',
 '2020.07.05',
 '2020.07.03',
 '2020.06.29',
 '2020.06.26',
 '2020.06.24',
 '2020.06.18',
 '2020.06.12',
 '2020.06.10',
 '2020.06.07',
 '2020.06.05',
 '2020.06.03',
 '2020.05.28',
 '2020.05.22',
 "'KODEX 200선물인버스2X'",
 '네이버앱의']

In [21]:
date_length=len(date2)

In [22]:
del date2[0]
del date2[0]
del date2[0]
del date2[0]
del date2[date_length-6]
del date2[date_length-6]# len 처리 다시. 뉴스개수가 다다른거 고려
date2 #앞뒤 쓰레기값 제거

['2020.10.11',
 '2020.09.27',
 '2020.08.09',
 '2020.08.02',
 '2020.07.16',
 '2020.07.14',
 '2020.07.12',
 '2020.07.05',
 '2020.07.03',
 '2020.06.29',
 '2020.06.26',
 '2020.06.24',
 '2020.06.18',
 '2020.06.12',
 '2020.06.10',
 '2020.06.07',
 '2020.06.05',
 '2020.06.03',
 '2020.05.28',
 '2020.05.22']

In [23]:
#저장형식을 알려주기위한 예시파일
os.chdir('D:\\ai\\소활코\\homework\\'+dir_name)
now = time.localtime()
txt2=' %04d-%02d-%02d-%02d-%02d-%02d' %(now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

dir_name2 = "[예시파일] "+stock_name[0]+txt2
os.mkdir(dir_name2)
time.sleep(0.3)
no2=[]
os.chdir('D:\\ai\\소활코\\homework\\'+dir_name+'\\'+dir_name2)
#이따 디렉토리 위치 마지막에 다시 복구하는거 까먹지않기
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
news_list = soup.select('ul > li')
news_f=stock_name[0]+"news"+txt2+".txt"
news_fc=stock_name[0]+"news"+txt2+".csv"
news_fx=stock_name[0]+"news"+txt2+".xlsx"
for i in range(len(title2)-1):
    f=open(news_f, 'a', encoding='UTF-8')
    f.write("["+str(i+1)+"]"+"\n")
    print("["+str(i+1)+"]")
    no2.append(i+1)
    #제목
    f.write("1. 제목 :"+title2[i]+"\n")
    print("1. 제목 :"+title2[i]+"\n")
    #내용
    f.write("2. 내용 :"+contents2[i]+"\n")
    print("2. 내용 :"+contents2[i]+"\n")
    #발행처
    f.write("3. 발행지 :"+news2[i]+"\n")
    print("3. 발행지 :"+news2[i]+"\n")
    #날짜
    f.write("4. 날짜 :"+date2[i]+"\n")
    print("4. 날짜 :"+date2[i]+"\n")
time.sleep(0.5)  



[1]
1. 제목 :[마켓포인트]코스피 주간 개인 순매수 1위 'KODEX 200 선물인버스 2X'

2. 내용 :조용석

3. 발행지 :이데일리

4. 날짜 :2020.10.11

[2]
1. 제목 :[마켓포인트]코스피 주간 기관 순매수 1위 `KODEX 200선물인버스 2X`

2. 내용 :마켓포인트 2671 최정희

3. 발행지 :이데일리

4. 날짜 :2020.09.27

[3]
1. 제목 :[마켓포인트]코스피 주간 기관 순매도 1위 ‘KODEX 200선물인버스2X’

2. 내용 :고준혁

3. 발행지 :이데일리

4. 날짜 :2020.08.09

[4]
1. 제목 :[마켓포인트]코스피 주간 개인 순매수 1위 ‘KODEX200선물인버스2X’

2. 내용 :이광수

3. 발행지 :이데일리

4. 날짜 :2020.08.02

[5]
1. 제목 :16일, 기관 거래소에서 KODEX 200선물인버스2X(+2.16%), 현대차(+0.92%) 등 순매수

2. 내용 :기관 투자자는 16일 거래소에서 KODEX 200선물인버스2X, 현대차, KT&G 등을 중점적으로 사들인 것으로 나타났다. 기관 투자자의 순매수 상위 20개 종목은 KODEX 200선물인버스2X, 현대차, KT&G, KODEX WTI원유선물(H), SK텔레콤, 한국금융지주, 기아차... 

3. 발행지 :한국경제

4. 날짜 :2020.07.16

[6]
1. 제목 :14일, 기관 거래소에서 LG전자(+3.36%), KODEX 200선물인버스2X(+0.2%) 등 순매수

2. 내용 :기관 투자자는 14일 거래소에서 LG전자, KODEX 200선물인버스2X, 씨에스윈드 등을 중점적으로 사들인 것으로 나타났다. 기관 투자자의 순매수 상위 20개 종목은 LG전자, KODEX 200선물인버스2X, 씨에스윈드, SK텔레콤, 금호석유, LG유플러스... 

3. 발행지 :한국경제

4. 날짜 :2020.07.14

[7]
1. 제목 :[마켓포인트]코스피 주간 개인 순매도 1위 ‘KODEX200선물인버스2X’



In [24]:
del title2[len(title2)-1] #pandas value error 방지

In [25]:
#pandas 활용하여 데이터 프레임 생성
stocknews_1 = pd.DataFrame()
stocknews_1['번호'] = no2
stocknews_1['제목'] = title2
stocknews_1['내용'] = contents2
stocknews_1['발행지'] = news2
stocknews_1['날짜'] = date2

In [26]:
#csv 형태로 저장하기
stocknews_1.to_csv(news_fc,encoding="utf-8-sig",index=False)

#엑셀 형태로 저장하기
stocknews_1.to_excel(news_fx,index=False)

In [27]:
#전체를 for문으로 크게 묶어서 셀을 나눌수 없었습니다.
n=1
for i in range(100):
    no2=[]           # 뉴스 번호 컬럼
    title2=[]       # 뉴스 제목 컬럼
    contents2=[]    # 뉴스 내용 컬럼
    news2=[] # 발행처
    date2=[] # 뉴스 날짜
    
    driver.find_element_by_class_name('Nicon_search').click()
    element = driver.find_element_by_class_name('Nbox_input_text')
    element.send_keys(stock_name[n]) #1~100순위 종목 순차적 입력
    time.sleep(0.5)
    #driver.find_element_by_class_name('Nicon_submit').click()
    #driver.find_element_by_link_text(stock_name[n]).click()
    #driver.find_element_by_class_name('stock_item').click() 
    driver.find_element_by_xpath("""//*[@id="a_link"]""").click()
    time.sleep(0.5)
    driver.find_element_by_link_text("뉴스·공시").click()
    
    html = driver.page_source
    #soup = BeautifulSoup(html, 'html.parser')
    news_list = soup.select('.title')
    news_paragraph=soup.select('.news_ct')
    news_maker=soup.select('.press')
    news_date=soup.select('em',class_='time')
    #print(news_list)
    for title in news_list:
        title2.append(title.get_text())
    for para in news_paragraph:
        contents2.append(para.get_text())
    for maker in news_maker:
        news2.append(maker.get_text())
    for date in news_date:
        date2.append(date.get_text())
    date_length=len(date2)
    del date2[0]
    del date2[0]
    del date2[0]
    del date2[0]
    del date2[date_length-6]
    del date2[date_length-6]
    
    #check for pandas value error
    print('◆◆◆◆◆')
    print(len(title2))
    print(len(contents2))
    print(len(news2))
    print(len(date2))
    
    os.chdir('D:\\ai\\소활코\\homework\\'+dir_name)
    now = time.localtime()
    txt2=' %04d-%02d-%02d-%02d-%02d-%02d' %(now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

    dir_name2 = str(n)+". "+stock_name[n-1]+txt2
    os.mkdir(dir_name2)
    time.sleep(0.3)
    os.chdir('D:\\ai\\소활코\\homework\\'+dir_name+'\\'+dir_name2)
    #이따 디렉토리 위치 마지막에 다시 복구하는거 까먹지않기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    news_list = soup.select('ul > li')
    news_f_1=stock_name[n-1]+"news"+txt2+".txt"
    news_fc=stock_name[n-1]+"news"+txt2+".csv"
    news_fx=stock_name[n-1]+"news"+txt2+".xlsx"
    try:
        for i in range(len(title2)-1):
            f=open(news_f_1, 'a', encoding='UTF-8')
            f.write("["+str(i+1)+"]"+"\n")
            print("["+str(i+1)+"]")
            no2.append(i)
            #제목
            f.write("1. 제목 :"+title2[i]+"\n")
            print("1. 제목 :"+title2[i]+"\n")
            #내용
            f.write("2. 내용 :"+contents2[i]+"\n")
            print("2. 내용 :"+contents2[i]+"\n")
            #발행처
            f.write("3. 발행지 :"+news2[i]+"\n")
            print("3. 발행지 :"+news2[i]+"\n")
            #날짜
            f.write("4. 날짜 :"+date2[i]+"\n")
            print("4. 날짜 :"+date2[i]+"\n")  
        time.sleep(0.3)
    except:
        del title2[len(title2)-1] #pandas value error 방지
        for i in range(len(title2)-1):
            f=open(news_f_1, 'a', encoding='UTF-8')
            f.write("["+str(i+1)+"]"+"\n")
            print("["+str(i+1)+"]")
            no2.append(i+1)
            #제목
            f.write("1. 제목 :"+title2[i]+"\n")
            print("1. 제목 :"+title2[i]+"\n")
            #내용
            f.write("2. 내용 :"+contents2[i]+"\n")
            print("2. 내용 :"+contents2[i]+"\n")
            #발행처
            f.write("3. 발행지 :"+news2[i]+"\n")
            print("3. 발행지 :"+news2[i]+"\n")
            #날짜
            f.write("4. 날짜 :"+date2[i]+"\n")
            print("4. 날짜 :"+date2[i]+"\n")  
        time.sleep(0.3)
        
    #pandas 활용하여 데이터 프레임 생성
    try:
        del title2[len(title2)-1] #pandas value error 방지
        stocknews_2 = pd.DataFrame()
        stocknews_2['번호'] = no2
        stocknews_2['제목'] = title2
        stocknews_2['내용'] = contents2
        stocknews_2['발행지'] = news2
        stocknews_2['날짜'] = date2
        time.sleep(0.3)
    except:
        try:
            #del title2[len(title2)-1] #pandas value error 방지
            del date2[len(date2)-1] #pandas value error 방지
            stocknews_2 = pd.DataFrame()
            stocknews_2['번호'] = no2
            stocknews_2['제목'] = title2
            stocknews_2['내용'] = contents2
            stocknews_2['발행지'] = news2
            stocknews_2['날짜'] = date2
            time.sleep(0.3)
        except:
            del date2[len(date2)-2] #pandas value error 방지
            stocknews_2 = pd.DataFrame()
            stocknews_2['번호'] = no2
            stocknews_2['제목'] = title2
            stocknews_2['내용'] = contents2
            stocknews_2['발행지'] = news2
            stocknews_2['날짜'] = date2
            time.sleep(0.3)
    #csv 형태로 저장하기
    stocknews_2.to_csv(news_fc,encoding="utf-8-sig",index=False)
    #엑셀 형태로 저장하기
    stocknews_2.to_excel(news_fx,index=False)
    n+=1

◆◆◆◆◆
21
20
20
20
[1]
1. 제목 :[마켓포인트]코스피 주간 개인 순매수 1위 'KODEX 200 선물인버스 2X'

2. 내용 :조용석

3. 발행지 :이데일리

4. 날짜 :2020.10.11

[2]
1. 제목 :[마켓포인트]코스피 주간 기관 순매수 1위 `KODEX 200선물인버스 2X`

2. 내용 :마켓포인트 2671 최정희

3. 발행지 :이데일리

4. 날짜 :2020.09.27

[3]
1. 제목 :[마켓포인트]코스피 주간 기관 순매도 1위 ‘KODEX 200선물인버스2X’

2. 내용 :고준혁

3. 발행지 :이데일리

4. 날짜 :2020.08.09

[4]
1. 제목 :[마켓포인트]코스피 주간 개인 순매수 1위 ‘KODEX200선물인버스2X’

2. 내용 :이광수

3. 발행지 :이데일리

4. 날짜 :2020.08.02

[5]
1. 제목 :16일, 기관 거래소에서 KODEX 200선물인버스2X(+2.16%), 현대차(+0.92%) 등 순매수

2. 내용 :기관 투자자는 16일 거래소에서 KODEX 200선물인버스2X, 현대차, KT&G 등을 중점적으로 사들인 것으로 나타났다. 기관 투자자의 순매수 상위 20개 종목은 KODEX 200선물인버스2X, 현대차, KT&G, KODEX WTI원유선물(H), SK텔레콤, 한국금융지주, 기아차... 

3. 발행지 :한국경제

4. 날짜 :2020.07.16

[6]
1. 제목 :14일, 기관 거래소에서 LG전자(+3.36%), KODEX 200선물인버스2X(+0.2%) 등 순매수

2. 내용 :기관 투자자는 14일 거래소에서 LG전자, KODEX 200선물인버스2X, 씨에스윈드 등을 중점적으로 사들인 것으로 나타났다. 기관 투자자의 순매수 상위 20개 종목은 LG전자, KODEX 200선물인버스2X, 씨에스윈드, SK텔레콤, 금호석유, LG유플러스... 

3. 발행지 :한국경제

4. 날짜 :2020.07.14

[7]
1. 제목 :[마켓포인트]코스피 주간 개인 순매도 1위 ‘

◆◆◆◆◆
21
20
20
20
[1]
1. 제목 :[마켓포인트]코스피 주간 기관 순매수 1위 'KODEX 레버리지'

2. 내용 :조용석

3. 발행지 :이데일리

4. 날짜 :2020.10.11

[2]
1. 제목 :[마켓포인트]코스피 주간 기관 순매수 1위 ‘KODEX레버리지’

2. 내용 :이광수

3. 발행지 :이데일리

4. 날짜 :2020.08.02

[3]
1. 제목 :[마켓포인트]코스피 주간 기관 순매수 1위 'KODEX 레버리지'

2. 내용 :△마켓포인트 화면번호 2671번 박종오

3. 발행지 :이데일리

4. 날짜 :2020.07.19

[4]
1. 제목 :13일, 기관 거래소에서 KODEX 레버리지(+3.84%), 삼성전자(+1.33%) 등 순매수

2. 내용 :기관 투자자는 13일 거래소에서 KODEX 레버리지, 삼성전자, KODEX WTI원유선물(H) 등을 중점적으로 사들인 것으로 나타났다. 기관 투자자의 순매수 상위 20개 종목은 KODEX 레버리지, 삼성전자, KODEX WTI원유선물(H), 현대모비스, KODEX... 

3. 발행지 :한국경제

4. 날짜 :2020.07.13

[5]
1. 제목 :[마켓포인트]코스피 주간 기관 순매수 1위 'KODEX 레버리지'

2. 내용 :김윤지

3. 발행지 :이데일리

4. 날짜 :2020.07.05

[6]
1. 제목 :02일, 기관 거래소에서 KODEX 레버리지(+2.4%), NAVER(+2.97%) 등 순매수

2. 내용 :기관 투자자는 02일 거래소에서 KODEX 레버리지, NAVER, 카카오 등을 중점적으로 사들인 것으로 나타났다. 기관 투자자의 순매수 상위 20개 종목은 KODEX 레버리지, NAVER, 카카오, SK텔레콤, LG화학, KODEX 코스닥150 레버리지, 삼성바이오로직스... 

3. 발행지 :한국경제

4. 날짜 :2020.07.02

[7]
1. 제목 :01일, 기관 거래소에서 TIGER 200(+0.05%), KODEX 레버리지(0.00%) 등 순매도



◆◆◆◆◆
20
19
19
19
[1]
1. 제목 :레버리지WTI원유ETN, 22일 거래 재개…"징검다리 거래 계속 이어진다"

2. 내용 :서부텍사스유(WTI)를 기초자산으로 하는 레버리지 원유 상장지수증권(ETN)이 또다시 거래정지 상태에 놓였다. 다음 거래일은 오는 22일이다. 18일 한국거래소에 따르면 ‘신한레버리지WTI원유ETN’, ‘삼성레버리지WT원유ETN’... 

3. 발행지 :아시아경제

4. 날짜 :2020.05.18

[2]
1. 제목 :레버리지WTI원유ETN, 내일부터 3거래일 매매정지

2. 내용 :서부텍사스유(WTI)를 기초자산으로 하는 레버리지 원유 상장지수증권(ETN)이 거래정지 상태에 놓이면서 오는 18일 거래가 재개될 예정이다. 12일 한국거래소에 따르면 ‘신한레버리지WTI원유ETN’, ‘삼성레버리지WT원유ETN’... 

3. 발행지 :아시아경제

4. 날짜 :2020.05.12

[3]
1. 제목 :삼성자산운용, WTI원유선물 ETF 6월물 →7월물 전부 교체

2. 내용 :삼성자산운용이 코덱스(KODEX) WTI원유선물 상장지수펀드(ETF) 보유 종목 중 WTI 원유선물 6월물을 7월물로 모두 변경했다고 30일 밝혔다. 삼성자산운용은 이 날 “지난 28일 공지한 바와 같이 KODEX WTI원유선물(H) ETF 펀드의 기초지수인... 

3. 발행지 :파이낸셜뉴스

4. 날짜 :2020.04.30

[4]
1. 제목 :삼성자산운용, WTI원유선물 ETF에서 6월물→7월물 교체

2. 내용 :삼성자산운용이 코덱스(KODEX) WTI원유선물 상장지수펀드(ETF) 보유 종목 중 WTI 원유선물 일부 6월물을 7월물로 변경했다고 29일 밝혔다. 삼성운용은 이날 "해당 ETF의 기초지수인 ‘S&P GCSI Crude Oil Index Excess Return’에서 구성종목인... 

3. 발행지 :파이낸셜뉴스

4. 날짜 :2020.04.29

[5]
1. 제목 :유가 변동성 확대에 WTI원유 ETN 급등락 지속

2. 내용 :국제유

◆◆◆◆◆
21
20
20
20
[1]
1. 제목 :[특징주] 남성, 아마존 전기차 공개...세계최초 개발 '알렉사' 탑재 소식에↑

2. 내용 :글로벌 전자상거래 업체 아마존이 인공지능(AI) 비서로 알려진 '알렉사'를 탑재한 전기승합차를 공개하면서 남성 등 관련주가 강세다. 12일 오후 1시 20분 현재 남성은 유가증권시장에서 전일 대비 7.77% 오른 3050원에 거래되고 있다. CNBC 등 외신에... 

3. 발행지 :파이낸셜뉴스

4. 날짜 :2020.10.12

[2]
1. 제목 :'남성' 5% 이상 상승, 단기·중기 이평선 정배열로 상승세

2. 내용 :◆ 차트 분석 - 단기·중기 이평선 정배열로 상승세 일봉 차트의 모습은 현재 단기·중기 이평선이 정배열 상태로 상승세를 보이고 있다. [그래프]남성 차트 분석 ◆ 주체별 매매동향 - 외국인과 개인의 거래비중 높음, 각각 10.2%, 83.7% (최근한달)... 

3. 발행지 :한국경제

4. 날짜 :2020.07.07

[3]
1. 제목 :'남성' 5% 이상 상승, 주가 상승세, 단기 이평선 역배열 구간

2. 내용 :◆ 차트 분석 - 주가 상승세, 단기 이평선 역배열 구간 추세선으로 보면 단기 이평선이 역배열을 만들며 조정을 받다가 다시 상승으로 돌아선 모습이다. 최근 이 종목의 차트에서 최근 5일 매물대 돌파, 주가 5MA 상향돌파, 최근 60일 매물대 돌파 등의... 

3. 발행지 :한국경제

4. 날짜 :2020.06.26

[4]
1. 제목 :'남성' 5% 이상 상승, 전일 기관 대량 순매수

2. 내용 :◆ 주체별 매매동향 - 전일 기관 대량 순매수 지난 한달을 기준으로 보면 기관이 16.5만주를 순매수했고, 개인들도 27.0만주를 순매수했다. 반면 그동안 외국인은 42.9만주를 순매도했다. 같은 기간 개인의 거래비중이 89.6%로 가장 높아, 주요 거래... 

3. 발행지 :한국경제

4. 날짜 :2020.06.15

[5]
1. 제목 :남성, +5.30% 상승폭 확대

2. 내

◆◆◆◆◆
21
20
20
20
[1]
1. 제목 :삼성엔지니어링, 멕시코서 4.5조 규모 수주

2. 내용 :삼성엔지니어링이 멕시코에서 4조5000억원에 달하는 초대형 프로젝트를 수주했다. 삼성엔지니어링은 멕시코 법인이 지난 28일(현지시간) 멕시코 국영석유회사인 페멕스(PEMEX)의 자회사 PTI-ID로부터 '도스보카스 정유 프로젝트' 패키지 2, 3의 설계... 

3. 발행지 :파이낸셜뉴스

4. 날짜 :2020.10.30

[2]
1. 제목 :삼성엔지니어링 4.5조원 수주 `잭팟`

2. 내용 :멕시코 원유생산설비 건설 창사후 최대…주가 13% ↑ 삼성엔지니어링이 멕시코에서 4조5000억원 규모의 초대형 프로젝트를 수주했다. 30일 삼성엔지니어링은 자사 멕시코법인이 28일(현지시간) 멕시코 국영석유회사인 페멕스의 자회사인 PTI... 

3. 발행지 :매일경제

4. 날짜 :2020.10.30

[3]
1. 제목 :[클릭 e종목] "삼성엔지니어링, 호실적 거뒀지만 신규 수주 필요"

2. 내용 :삼성엔지니어링이 올해 3분기 시장전망치(컨센서스)를 뛰어넘는 실적을 기록했다. 다만 아직 신규 수주 실적이 연간 목표에 한참 못 미치는 만큼 추가 수주가 절실하다는 분석이다. 30일 유안타증권은 삼성엔지니어링에 대해 이 같이 분석했다.... 

3. 발행지 :아시아경제

4. 날짜 :2020.10.30

[4]
1. 제목 :삼성엔지니어링, 3분기 영업익 1000억..전년比 0.3%↑

2. 내용 :삼성엔지니어링(028050)은 지난 3분기 연결재무제표 기준 영업이익이 1000억원으로 전년 동기 대비 0.3% 증가했다고 29일 공시했다. 매출액은 1조6032억원으로 전년 동기 대비 2.0% 감소했으며, 당기순이익은 596억원으로 전년 동기 대비 15.9... 

3. 발행지 :이데일리

4. 날짜 :2020.10.29

[5]
1. 제목 :삼성엔지니어링, 3분기 영업익 1001억원…작년 대비 0.3%↑

2. 내용 :삼성엔지니어링은 29일 공시를 통해 2020년

...

◆◆◆◆◆
21
20
20
20
[1]
1. 제목 :한농화성, “현저한 시황변동 사유없다”

2. 내용 :한농화성은 최근 현저한 시황변동 관련 조회공시 답변으로 “최근 현저한 시황변동에 영향을 미칠만한 사항으로써 진행중이거나 중요한 공시사항이 없다”고 12일 공시했다.

3. 발행지 :헤럴드경제

4. 날짜 :2020.10.12

[2]
1. 제목 :한농화성, 시황변동 조회공시 요구

2. 내용 :한국거래소는 한농화성(011500)에 시황변동 관련 조회공시를 요구했다. 공시시한은 오는 12일 오후 6시다. 함지현

3. 발행지 :이데일리

4. 날짜 :2020.10.08

[3]
1. 제목 :한농화성, 테슬라 배터리데이 요구 해법…`전고체 배터리` 핵심소재 개발 부각

2. 내용 :LG화학·한국화학연구원 등과 국책과제 수행 글로벌 전기차 업체 테슬라가 배터리 업계에 원가절감과 성능 개선을 공개적으로 요청하면서 국내 2차전지 업계 발등에도 불이 떨어졌다. 이 가운데 정밀화학 기업 한농화성이 LG화학, 한국화학연구원 등과... 

3. 발행지 :매일경제

4. 날짜 :2020.09.23

[4]
1. 제목 :'한농화성' 5% 이상 상승, 단기·중기 이평선 정배열로 상승세

2. 내용 :◆ 차트 분석 - 단기·중기 이평선 정배열로 상승세 이 시간 차트의 흐름은 현재 단기·중기 이평선이 정배열 상태로 상승세를 보이고 있다. [그래프]한농화성 차트 분석 ◆ 기업개요 한농화성은 글리콜에테르 생산 등 정밀화학 제조업체로 알려져 있다.... 

3. 발행지 :한국경제

4. 날짜 :2020.05.04

[5]
1. 제목 :'한농화성' 5% 이상 상승, 주가 상승 중, 단기간 골든크로스 형성

2. 내용 :◆ 차트 분석 - 주가 상승 중, 단기간 골든크로스 형성 추세선으로 보면 단기간에 골든크로스를 만들고 주가가 60일선까지 뚫고 가는 모습이다. 이 종목의 차트에서 최근에 최근 5일 매물대 돌파, 최근 60일 매물대 돌파 등을 확인할 수 있다 [그래프]... 

3. 발행지 

◆◆◆◆◆
13
12
12
12
[1]
1. 제목 :광전자, 현저한 시황변동 관련 "11월 중 분기보고서 공시예정 외 별도 공시사항 없음"

2. 내용 :광전자는 현저한 시황변동과 관련한 한국거래소의 조회공시 요구에 대해 "11월 중 분기보고서 공시가 예정되어 있다"며 이 외에 현재 진행 중이거나 확정된 공시사항이 없다고 22일 답변했다.

3. 발행지 :아시아경제

4. 날짜 :2020.10.22

[2]
1. 제목 :광전자, 현저한 시황변동 조회공시 요구

2. 내용 :한국거래소 유가증권시장본부는 광전자(017900)에 대해 현저한 시황변동에 대한 조회공시를 요구했다. 공시시한은 22일 오후 6시다. 노희준

3. 발행지 :이데일리

4. 날짜 :2020.10.21

[3]
1. 제목 :'광전자' 5% 이상 상승, 주가 20일 이평선 상회, 단기·중기 이평선 역배열

2. 내용 :◆ 차트 분석 - 주가 20일 이평선 상회, 단기·중기 이평선 역배열 일봉 차트의 모습은 중기적으로 역배열을 보이고 있지만 단기적으로 반등흐름이 나오면서 20일선을 넘어서는 모습이다. 이 종목의 차트에서 최근에 삼선전환도(양전환), 최근 5일... 

3. 발행지 :한국경제

4. 날짜 :2020.07.06

[4]
1. 제목 :[fnRASSI]광전자, 6.12% 내리며 거래량 증가

2. 내용 :광전자(017900)의 당일 거래량이 최근 5거래일 거래량의 최고치를 경신했다. 그리고 현재 6.12% 내린 1,765원에 거래중이며 4일 연속 하락흐름을 보이고 있다. 외국인 보유량 증가, 기관 감소 (5일 누적) 최근 5일간 동종목의 외국인 투자자는 1... 

3. 발행지 :파이낸셜뉴스

4. 날짜 :2020.06.12

[5]
1. 제목 :[fnRASSI]광전자, 1.26% 오르며 거래량 증가

2. 내용 :광전자(017900)는 당일 거래량 증가로 최근 5거래일 거래량의 최고치를 경신했다. 그리고 광전자는 4일 연속 상승흐름을 보이며, 11시 28분 현재 1.26% 증가한 2,0

◆◆◆◆◆
21
20
20
20
[1]
1. 제목 :KT&G, 청각장애인 소통 위한 투명 마스크 지원

2. 내용 :KT&G가 코로나19 확산으로 소통에 어려움을 겪고 있는 청각장애인을 돕기 위해 투명 마스크 후원 캠페인에 동참한다. KT&G는 28일 서울 중구 사랑의달팽이 본사에서 이상학 KT&G 지속경영본부장과김민자 사랑의달팽이 회장이 참석한 가운데... 

3. 발행지 :파이낸셜뉴스

4. 날짜 :2020.10.29

[2]
1. 제목 :포스코·LG전자·KT&G…내년 날개펼칠 저평가株

2. 내용 :韓GDP·코스피 순이익 추정치 코로나 이전 수준으로 회복돼 내년 글로벌 경제회복 가시권 포스코·KT&G…수출株 포함 고배당50지수 이달 3% 상회 LG전자·대한유화 업황회복에 내년 영업익 50%대 상승 기대 현재 저평가된 가치주가 내년에 반등할 수... 

3. 발행지 :매일경제

4. 날짜 :2020.10.28

[3]
1. 제목 :KT&G `릴 하이브리드 2.0`, 전자담배 메카 `일본` 공략

2. 내용 :KT&G(사장 백복인)는 필립모리스 인터내셔널(PMI)과 글로벌 협업을 통해 일본에 궐련형 전자담배 '릴 하이브리드 2.0(lil Hybrid 2.0)' 및 전용스틱 '믹스(MIIX)'를 26일부터 판매한다고 21일 밝혔다. 러시아·우크라이나에 이어 3번째 출시국가인... 

3. 발행지 :매일경제

4. 날짜 :2020.10.21

[4]
1. 제목 :KT&G 전자담배 일본 간다…`릴 하이브리드 2.0` 첫 출시

2. 내용 :필립모리스와 글로벌 협업 해외시장 공략 본격화 나서 KT&G(대표 백복인)가 일본시장에서 글로벌 전자담배 제조사들과 승부를 벌인다. KT&G는 오는 26일 필립모리스인터내셔널(PMI)과 글로벌 협업을 통해 일본에 궐련형 전자담배 '릴 하이브리드 2.0... 

3. 발행지 :매일경제

4. 날짜 :2020.10.21

[5]
1. 제목 :KT&G '릴 솔리드 2.0', 전국 주요 도시로 판매처 확대

2. 내용 :KT&G는 오는

◆◆◆◆◆
21
20
20
20
[1]
1. 제목 :'빅히트 소나기' 맞은 엔터3사…넷마블·디피씨도 ↓

2. 내용 :[빅히트 지분 보유한 넷마블·디피씨는 각각 10, 20% 급락] 빅히트엔터테인먼트가 상장 첫날 예상과 달리 시초가를 하회하면서 엔터주들이 동반 하락했다. 빅히트 지분을 가진 회사들도 소나기를 피하지 못했다. '뉴스에 팔아라'는 증시... 

3. 발행지 :머니투데이

4. 날짜 :2020.10.16

[2]
1. 제목 :빅히트 재료 소멸…넷마블·디피씨 나란히 ↓

2. 내용 :빅히트엔터테인먼트 공모주 청약 마지막날 넷마블과 디피씨가 하락세다. 그동안 빅히트 상장 기대감에 주가가 올라왔다가 재료가 소멸하면서 이틀째 약세를 보이고 있는 것으로 풀이된다. 6일 오전 9시19분 넷마블은 전일대비 7000원(4.28... 

3. 발행지 :머니투데이

4. 날짜 :2020.10.06

[3]
1. 제목 :BTS 빌보드 '핫 100' 1위에 디피씨·초록뱀 등 관련주 '급등'

2. 내용 :그룹 방탄소년단(BTS)이 한국 가수 가운데 처음으로 빌보드 메인 싱글 차트인 '핫 100' 정상에 오르면서 관련주가 동반 강세를 보이고 있다. 1일 오전 9시2분 현재 디피씨는 전 거래일보다 2400원(12.63%) 급등한 2만1400원에 거래되고 있다.... 

3. 발행지 :한국경제

4. 날짜 :2020.09.01

[4]
1. 제목 :[시그널] 3개월만에 2.6배↑···디피씨, 자기주식 50만주 처분

2. 내용 :디피씨(026890)가 93억원어치 자기주식을 처분한다. 디피씨는 처분 목적을 운영자금 확충이라고 밝혔지만 주가 급등 역시 처분에 나선 계기로 보인다. 디피씨가 보통주 50만주를 주당 1만 8,600원에 장내에서 처분하기로 결정했다고 21일... 

3. 발행지 :서울경제

4. 날짜 :2020.07.21

[5]
1. 제목 :디피씨, 93억원 규모 자사주 처분 결정

2. 내용 :디피씨(026890)는 운영자금 및 재무구조 개선을 위해 자사주 50만

IndexError: list index out of range

In [ ]:
driver.close( )

파이썬 문법이 익숙치 않은 탓도 있었지만 상상치도 못했던 에러와 오작동이 계속해 발생하는 것을 보았다. 구글링을 통해 해결되는 에러도 많았지만 그렇지 못한 것들도 많았고 특히 오작동의 경우에는 직접 경험해보지 않고서는 해결할 수 없는 것들이었다. 경험의 중요성에 대해 느꼈다. 